In [0]:
! pip install pyLDAvis

In [0]:
import numpy as np
import pandas as pd

In [0]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Latentt Dirilecht Allocation Model (LDA)

In [0]:
data = pd.read_csv("./review_labeled.csv")

In [0]:
data_reviews_0 = data['reviews.text']
data_reviews = data_reviews_0.values.tolist()

In [0]:
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

In [0]:
import nltk
nltk.download('stopwords')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Remove new line characters
data_reviews = [re.sub('\s+', ' ', sent) for sent in data_reviews]

# Remove distracting single quotes
data_reviews = [re.sub("\'", "", sent) for sent in data_reviews]

In [0]:
print(data_reviews[:1])

['Our experience at Rancho Valencia was absolutely perfect from beginning to end!!!! We felt special and very happy during our stayed. I would come back in a heart beat!!!']


In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data_reviews))

In [0]:
print(data_words[:1])


[['our', 'experience', 'at', 'rancho', 'valencia', 'was', 'absolutely', 'perfect', 'from', 'beginning', 'to', 'end', 'we', 'felt', 'special', 'and', 'very', 'happy', 'during', 'our', 'stayed', 'would', 'come', 'back', 'in', 'heart', 'beat']]


In [0]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['our', 'experience', 'at', 'rancho', 'valencia', 'was', 'absolutely', 'perfect', 'from', 'beginning', 'to', 'end', 'we', 'felt', 'special', 'and', 'very', 'happy', 'during', 'our', 'stayed', 'would', 'come', 'back', 'in', 'heart', 'beat']


In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['experience', 'rancho', 'valencia', 'absolutely', 'perfect', 'beginning', 'end', 'feel', 'special', 'happy', 'stay', 'would', 'come', 'back', 'heart', 'beat']]


In [0]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]]


In [0]:
id2word[4]

'come'

In [0]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolutely', 1),
  ('back', 1),
  ('beat', 1),
  ('beginning', 1),
  ('come', 1),
  ('end', 1),
  ('experience', 1),
  ('feel', 1),
  ('happy', 1),
  ('heart', 1),
  ('perfect', 1),
  ('rancho', 1),
  ('special', 1),
  ('stay', 1),
  ('valencia', 1),
  ('would', 1)]]

In [0]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [0]:
# Print the Keyword in the 9 topics
print(lda.print_topics())
doc_lda = lda[corpus]

[(0, '0.054*"check" + 0.038*"front_desk" + 0.029*"experience" + 0.019*"tell" + 0.018*"people" + 0.017*"ask" + 0.016*"arrive" + 0.016*"hour" + 0.014*"drive" + 0.013*"wait"'), (1, '0.074*"room" + 0.058*"hotel" + 0.038*"staff" + 0.033*"good" + 0.032*"clean" + 0.031*"breakfast" + 0.031*"great" + 0.025*"nice" + 0.019*"would" + 0.018*"get"'), (2, '0.052*"restaurant" + 0.052*"close" + 0.040*"also" + 0.026*"view" + 0.025*"suite" + 0.022*"offer" + 0.020*"quiet" + 0.018*"pleasant" + 0.016*"morning" + 0.015*"buffet"'), (3, '0.044*"call" + 0.036*"hot" + 0.028*"run" + 0.022*"cold" + 0.020*"available" + 0.019*"s" + 0.017*"continental" + 0.017*"manager" + 0.011*"move" + 0.011*"item"'), (4, '0.054*"family" + 0.047*"food" + 0.033*"always" + 0.021*"travel" + 0.018*"never" + 0.018*"business" + 0.016*"use" + 0.016*"spend" + 0.015*"value" + 0.015*"meet"'), (5, '0.052*"convenient" + 0.021*"full" + 0.021*"wifi" + 0.018*"unit" + 0.018*"microwave" + 0.017*"receive" + 0.016*"space" + 0.013*"indoor_pool" + 0.013

In [0]:
data['reviews.text'][3]

"Currently in bed writing this for the past hr 1/2 there have been dogs barking and squealing call the front desk to advise basically to be told there's nothing they can do. 315.00 and I can't sleep."

In [0]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.369266 -0.170814       1        1  39.088856
6      0.241810  0.226115       2        1  19.448236
0     -0.010511  0.130119       3        1   9.035082
2     -0.087391  0.096751       4        1   7.525023
7     -0.054866 -0.251620       5        1   6.513520
4     -0.102598  0.041839       6        1   6.179352
8     -0.100467 -0.017567       7        1   5.476983
3     -0.126123 -0.044930       8        1   3.959009
5     -0.129120 -0.009892       9        1   2.773941, topic_info=     Category         Freq           Term        Total  loglift  logprob
58    Default  4553.000000           room  4553.000000  30.0000  30.0000
13    Default  3270.000000           stay  3270.000000  29.0000  29.0000
76    Default  1180.000000            not  1180.000000  28.0000  28.0000
107   Default  3998.000000          hotel  3998.000000  27.0000  27.0000
61    Default  2313.000000          staff  2313.000000  26.0000  26.0000
116   Default  2044.000000           good  2044.000000  25.0000  25.0000
416   Default   780.000000          check   780.000000  24.0000  24.0000
133   Default  1992.000000          clean  1992.000000  23.0000  23.0000
25    Default  1926.000000          great  1926.000000  22.0000  22.0000
114   Default  1936.000000      breakfast  1936.000000  21.0000  21.0000
499   Default   583.000000           pool   583.000000  20.0000  20.0000
57    Default   620.000000     restaurant   620.000000  19.0000  19.0000
782   Default   612.000000          close   612.000000  18.0000  18.0000
24    Default  1034.000000             go  1034.000000  17.0000  17.0000
361   Default   632.000000             do   632.000000  16.0000  16.0000
105   Default   525.000000         family   525.000000  15.0000  15.0000
123   Default  1562.000000           nice  1562.000000  14.0000  14.0000
75    Default   592.000000     front_desk   592.000000  13.0000  13.0000
167   Default   994.000000           time   994.000000  12.0000  12.0000
494   Default   458.000000           food   458.000000  11.0000  11.0000
222   Default   474.000000           also   474.000000  10.0000  10.0000
6     Default   446.000000     experience   446.000000   9.0000   9.0000
120   Default   602.000000           make   602.000000   8.0000   8.0000
176   Default  1052.000000       friendly  1052.000000   7.0000   7.0000
312   Default   311.000000     everything   311.000000   6.0000   6.0000
71    Default   277.000000           call   277.000000   5.0000   5.0000
281   Default   322.000000         always   322.000000   4.0000   4.0000
195   Default   956.000000    comfortable   956.000000   3.0000   3.0000
31    Default   519.000000           take   519.000000   2.0000   2.0000
135   Default   912.000000       location   912.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
261    Topic9    90.519150           wifi    91.518791   3.5739  -3.8776
280    Topic9    79.393562           unit    80.393303   3.5724  -4.0087
614    Topic9    78.495750      microwave    79.495438   3.5722  -4.0201
362    Topic9    75.534248        receive    76.534576   3.5717  -4.0586
166    Topic9    71.157417          space    72.157089   3.5710  -4.1183
4605   Topic9    58.919147    indoor_pool    59.919365   3.5681  -4.3070
254    Topic9    58.003696          build    59.003384   3.5678  -4.3227
1463   Topic9    53.469093      condition    54.469086   3.5664  -4.4041
1062   Topic9    46.747784  uncomfortable    47.747715   3.5637  -4.5384
255    Topic9    45.738781          couch    46.738804   3.5633  -4.5602
3750   Topic9    45.411469    kitchenette    46.411797   3.5631  -4.5674
197    Topic9    38.578640          decor    39.578251   3.5593  -4.7305
210    Topic9    34.827854           talk    35.828209   3.5566  -4.8327
1188   Topic9    33.797676     atmosphere    34.797279   3.5558  -4.8628
203  